In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
import time
from datetime import datetime
from functools import reduce
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [3]:
#Query data from order_logs_snapshot

q = """
select order_id, accept_to_pickup_distance, captain_location_hex_8, captain_location_latitude,
captain_location_longitude, city_name, customer_location_hex_8, drop_location_hex_8, drop_location_latitude, drop_location_longitude, customer_location_latitude, 
customer_location_longitude, hhmmss, hour, quarter_hour, service_obj_service_name, time_bucket, weekday, yyyymmdd,
order_status, cancel_reason, updated_epoch, event_type
from orders.order_logs_immutable
where yyyymmdd >= '20220919'
and yyyymmdd <= '20221016'
and city_name in ('Bangalore')
and service_obj_service_name = 'Link'
and event_type in ('customer_cancelled', 'order_create', 'accepted')
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [4]:
#Load and view dataset
df = df_copy.copy()
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type
0,634347c23f0a16434178dd81,NaN,,NaN,NaN,Bangalore,8861892e89fffff,8861892537fffff,12.960986,77.638732,12.982827,77.637596,034426,03,0330,Link,03:59,1,20221010,new,None,1665353666852,order_create
1,6343487cb4d7b77788e4debd,NaN,,NaN,NaN,Bangalore,886189262bfffff,886189260dfffff,12.836120,77.669101,12.851847,77.664078,034732,03,0345,Link,03:59,1,20221010,new,None,1665353853367,order_create
2,634348d379e54f3e6056b663,NaN,,NaN,NaN,Bangalore,8860145b69fffff,8861892e1dfffff,12.985598,77.678218,12.993127,77.598427,034859,03,0345,Link,03:59,1,20221010,new,None,1665353940381,order_create
3,634341155a48c01d2a6828be,NaN,,NaN,NaN,Bangalore,88618920b1fffff,88618925cbfffff,12.922790,77.614345,12.956988,77.700621,031557,03,0315,Link,03:00,1,20221010,new,None,1665351958056,order_create
4,634343fa79e54f3e6056b440,NaN,,NaN,NaN,Bangalore,88618924a1fffff,88618924a7fffff,12.921461,77.571426,12.908926,77.573006,032818,03,0315,Link,03:00,1,20221010,new,None,1665352698322,order_create


In [5]:
#Create a column to identify OCARA, COBRA and COBRM
df['new_cancel_reason'] = np.where(df['event_type'] == 'customer_cancelled', np.where(df['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
    np.where(df['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')
df.head()

#Create a timestamp column from the epoch column
df['timestamp'] = df['updated_epoch'].apply(lambda x: datetime.utcfromtimestamp(x/1000))
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,new_cancel_reason,timestamp
0,634347c23f0a16434178dd81,NaN,,NaN,NaN,Bangalore,8861892e89fffff,8861892537fffff,12.960986,77.638732,12.982827,77.637596,034426,03,0330,Link,03:59,1,20221010,new,None,1665353666852,order_create,no_cancellation,2022-10-09 22:14:26.852
1,6343487cb4d7b77788e4debd,NaN,,NaN,NaN,Bangalore,886189262bfffff,886189260dfffff,12.836120,77.669101,12.851847,77.664078,034732,03,0345,Link,03:59,1,20221010,new,None,1665353853367,order_create,no_cancellation,2022-10-09 22:17:33.367
2,634348d379e54f3e6056b663,NaN,,NaN,NaN,Bangalore,8860145b69fffff,8861892e1dfffff,12.985598,77.678218,12.993127,77.598427,034859,03,0345,Link,03:59,1,20221010,new,None,1665353940381,order_create,no_cancellation,2022-10-09 22:19:00.381
3,634341155a48c01d2a6828be,NaN,,NaN,NaN,Bangalore,88618920b1fffff,88618925cbfffff,12.922790,77.614345,12.956988,77.700621,031557,03,0315,Link,03:00,1,20221010,new,None,1665351958056,order_create,no_cancellation,2022-10-09 21:45:58.056
4,634343fa79e54f3e6056b440,NaN,,NaN,NaN,Bangalore,88618924a1fffff,88618924a7fffff,12.921461,77.571426,12.908926,77.573006,032818,03,0315,Link,03:00,1,20221010,new,None,1665352698322,order_create,no_cancellation,2022-10-09 21:58:18.322


In [6]:
#Identify hexes in the high demand areas
bangalore_zone1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone1.csv')
bangalore_zone2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone2.csv')

bangalore_zone1_hexes = list(bangalore_zone1['customer_location_hex_8'].unique())
bangalore_zone2_hexes = list(bangalore_zone2['customer_location_hex_8'].unique())

df['zone_name'] = np.where(df['customer_location_hex_8'].isin(bangalore_zone1_hexes), 'zone_1',
     np.where(df['customer_location_hex_8'].isin(bangalore_zone2_hexes), 'zone_2', 'rest_of_bangalore'))
df.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,drop_location_hex_8,drop_location_latitude,drop_location_longitude,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,updated_epoch,event_type,new_cancel_reason,timestamp,zone_name
0,634347c23f0a16434178dd81,NaN,,NaN,NaN,Bangalore,8861892e89fffff,8861892537fffff,12.960986,77.638732,12.982827,77.637596,034426,03,0330,Link,03:59,1,20221010,new,None,1665353666852,order_create,no_cancellation,2022-10-09 22:14:26.852,rest_of_bangalore
1,6343487cb4d7b77788e4debd,NaN,,NaN,NaN,Bangalore,886189262bfffff,886189260dfffff,12.836120,77.669101,12.851847,77.664078,034732,03,0345,Link,03:59,1,20221010,new,None,1665353853367,order_create,no_cancellation,2022-10-09 22:17:33.367,rest_of_bangalore
2,634348d379e54f3e6056b663,NaN,,NaN,NaN,Bangalore,8860145b69fffff,8861892e1dfffff,12.985598,77.678218,12.993127,77.598427,034859,03,0345,Link,03:59,1,20221010,new,None,1665353940381,order_create,no_cancellation,2022-10-09 22:19:00.381,rest_of_bangalore
3,634341155a48c01d2a6828be,NaN,,NaN,NaN,Bangalore,88618920b1fffff,88618925cbfffff,12.922790,77.614345,12.956988,77.700621,031557,03,0315,Link,03:00,1,20221010,new,None,1665351958056,order_create,no_cancellation,2022-10-09 21:45:58.056,zone_2
4,634343fa79e54f3e6056b440,NaN,,NaN,NaN,Bangalore,88618924a1fffff,88618924a7fffff,12.921461,77.571426,12.908926,77.573006,032818,03,0315,Link,03:00,1,20221010,new,None,1665352698322,order_create,no_cancellation,2022-10-09 21:58:18.322,rest_of_bangalore


In [7]:
#Filter df on the columns needed
df_filter = df[['order_id', 'customer_location_hex_8', 'hour', 'yyyymmdd', 'timestamp', 'event_type', 'new_cancel_reason', 'zone_name']]

#Filter on the respective order_status
df_cobra = df_filter[df_filter['new_cancel_reason'] == 'COBRA']
df_ocara = df_filter[df_filter['new_cancel_reason'] == 'OCARA']
df_cobrm = df_filter[df_filter['new_cancel_reason'] == 'COBRM']
df_order_created = df_filter[df_filter['event_type'] == 'order_create']
df_accepted = df_filter[df_filter['event_type'] == 'accepted']

#Sort the dfs by timestamp
df_cobra = df_cobra.sort_values(by = 'timestamp', ascending=True)
df_cobra = df_cobra.drop_duplicates(subset='order_id')

df_ocara = df_ocara.sort_values(by = 'timestamp', ascending=True)
df_ocara = df_ocara.drop_duplicates(subset='order_id')

df_cobrm = df_cobrm.sort_values(by = 'timestamp', ascending=True)
df_cobrm = df_cobrm.drop_duplicates(subset='order_id')

df_accepted = df_accepted.sort_values(by = 'timestamp', ascending=True)
df_accepted = df_accepted.drop_duplicates(subset='order_id')

#Change the column names
df_cobra = df_cobra[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobra.columns = ['order_id', 'COBRA', 'timestamp_cobra']

df_cobrm = df_cobrm[['order_id', 'new_cancel_reason', 'timestamp']]
df_cobrm.columns = ['order_id', 'COBRM', 'timestamp_cobrm']

df_ocara = df_ocara[['order_id', 'new_cancel_reason', 'timestamp']]
df_ocara.columns = ['order_id', 'OCARA', 'timestamp_ocara']

df_accepted = df_accepted[['order_id', 'event_type', 'timestamp']]
df_accepted.columns = ['order_id', 'Accepted', 'timestamp_accepted']

#Merge the dfs together
dfs_to_merge = [df_order_created, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge).fillna('NA')
df_final.head()

,order_id,customer_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,zone_name,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted
0,634347c23f0a16434178dd81,8861892e89fffff,03,20221010,2022-10-09 22:14:26.852,order_create,no_cancellation,rest_of_bangalore,NA,NA,NA,NA,NA,NA,accepted,2022-10-09 22:15:20.039000
1,6343487cb4d7b77788e4debd,886189262bfffff,03,20221010,2022-10-09 22:17:33.367,order_create,no_cancellation,rest_of_bangalore,COBRA,2022-10-09 22:18:48.497000,NA,NA,NA,NA,NA,NA
2,634348d379e54f3e6056b663,8860145b69fffff,03,20221010,2022-10-09 22:19:00.381,order_create,no_cancellation,rest_of_bangalore,NA,NA,NA,NA,OCARA,2022-10-09 22:36:07.840000,accepted,2022-10-09 22:20:41.812000
3,634341155a48c01d2a6828be,88618920b1fffff,03,20221010,2022-10-09 21:45:58.056,order_create,no_cancellation,zone_2,COBRA,2022-10-09 21:46:39.015000,NA,NA,NA,NA,NA,NA
4,634343fa79e54f3e6056b440,88618924a1fffff,03,20221010,2022-10-09 21:58:18.322,order_create,no_cancellation,rest_of_bangalore,COBRA,2022-10-09 21:58:49.158000,NA,NA,NA,NA,NA,NA


In [8]:
#Filter on the respective order_status
df_cobra = df_final[df_final['COBRA'] == 'COBRA']
df_ocara = df_final[df_final['OCARA'] == 'OCARA']
df_cobrm = df_final[df_final['COBRM'] == 'COBRM']
df_accepted = df_final[df_final['Accepted'] == 'accepted']

#Calculate TTC for cobrm, ocara, and cobra and tta
df_cobra['ttc_cobra'] = pd.to_datetime(df_cobra['timestamp_cobra']) - pd.to_datetime(df_cobra['timestamp'])
df_cobra['ttc_cobra'] = df_cobra['ttc_cobra'].apply(lambda x: x.total_seconds())

df_cobrm['ttc_cobrm'] = pd.to_datetime(df_cobrm['timestamp_cobrm']) - pd.to_datetime(df_cobrm['timestamp'])
df_cobrm['ttc_cobrm'] = df_cobrm['ttc_cobrm'].apply(lambda x: x.total_seconds())

df_ocara['ttc_ocara'] = pd.to_datetime(df_ocara['timestamp_ocara']) - pd.to_datetime(df_ocara['timestamp'])
df_ocara['ttc_ocara'] = df_ocara['ttc_ocara'].apply(lambda x: x.total_seconds())

df_accepted['tta'] = pd.to_datetime(df_accepted['timestamp_accepted']) - pd.to_datetime(df_accepted['timestamp'])
df_accepted['tta'] = df_accepted['tta'].apply(lambda x: x.total_seconds())

#Filter only on the new columns made
df_cobra = df_cobra[['order_id', 'ttc_cobra']]
df_cobrm = df_cobrm[['order_id', 'ttc_cobrm']]
df_ocara = df_ocara[['order_id', 'ttc_ocara']]
df_accepted = df_accepted[['order_id', 'tta']]

#Merge the dfs together
dfs_to_merge = [df_final, df_cobra, df_cobrm, df_ocara, df_accepted]
df_final = reduce(lambda  left, right: pd.merge(left, right,on=['order_id'], how='left'), dfs_to_merge).fillna('NA')
df_final.head()

,order_id,customer_location_hex_8,hour,yyyymmdd,timestamp,event_type,new_cancel_reason,zone_name,COBRA,timestamp_cobra,COBRM,timestamp_cobrm,OCARA,timestamp_ocara,Accepted,timestamp_accepted,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,634347c23f0a16434178dd81,8861892e89fffff,03,20221010,2022-10-09 22:14:26.852,order_create,no_cancellation,rest_of_bangalore,NA,NA,NA,NA,NA,NA,accepted,2022-10-09 22:15:20.039000,NA,NA,NA,53.187
1,6343487cb4d7b77788e4debd,886189262bfffff,03,20221010,2022-10-09 22:17:33.367,order_create,no_cancellation,rest_of_bangalore,COBRA,2022-10-09 22:18:48.497000,NA,NA,NA,NA,NA,NA,75.13,NA,NA,NA
2,634348d379e54f3e6056b663,8860145b69fffff,03,20221010,2022-10-09 22:19:00.381,order_create,no_cancellation,rest_of_bangalore,NA,NA,NA,NA,OCARA,2022-10-09 22:36:07.840000,accepted,2022-10-09 22:20:41.812000,NA,NA,1027.459,101.431
3,634341155a48c01d2a6828be,88618920b1fffff,03,20221010,2022-10-09 21:45:58.056,order_create,no_cancellation,zone_2,COBRA,2022-10-09 21:46:39.015000,NA,NA,NA,NA,NA,NA,40.959,NA,NA,NA
4,634343fa79e54f3e6056b440,88618924a1fffff,03,20221010,2022-10-09 21:58:18.322,order_create,no_cancellation,rest_of_bangalore,COBRA,2022-10-09 21:58:49.158000,NA,NA,NA,NA,NA,NA,30.836,NA,NA,NA


In [9]:
#Create a date column
df_final['date'] = df_final['yyyymmdd'].apply(lambda x: x[6:])
df_final['count'] = 1

#Get average ttc, tta
df_cobra_grouped = df_final[df_final['COBRA'] == 'COBRA']
df_cobra_grouped = df_cobra_grouped.groupby(['zone_name', 'date', 'hour'])['ttc_cobra'].median().reset_index()
df_ocara_grouped = df_final[df_final['OCARA'] == 'OCARA']
df_ocara_grouped = df_ocara_grouped.groupby(['zone_name', 'date', 'hour'])['ttc_ocara'].median().reset_index()
df_cobrm_grouped = df_final[df_final['COBRM'] == 'COBRM']
df_cobrm_grouped = df_cobrm_grouped.groupby(['zone_name', 'date', 'hour'])['ttc_cobrm'].median().reset_index()
df_accepted_grouped = df_final[df_final['Accepted'] == 'accepted']
df_accepted_grouped = df_accepted_grouped.groupby(['zone_name', 'date', 'hour'])['tta'].median().reset_index()

#Merge the dfs together
dfs_to_merge = [df_cobra_grouped, df_cobrm_grouped, df_ocara_grouped, df_accepted_grouped]
df_metrics_final = reduce(lambda  left, right: pd.merge(left, right,on=['zone_name', 'date', 'hour'], how='left'), dfs_to_merge).fillna('NA')
df_metrics_final.head()

,zone_name,date,hour,ttc_cobra,ttc_cobrm,ttc_ocara,tta
0,rest_of_bangalore,01,00,93.9880,NA,347.6425,50.3110
1,rest_of_bangalore,01,01,89.6810,NA,290.2210,33.6045
2,rest_of_bangalore,01,02,88.0310,NA,309.5390,33.7775
3,rest_of_bangalore,01,03,72.0280,NA,357.8090,41.2755
4,rest_of_bangalore,01,04,87.3065,NA,294.3970,32.8745


In [11]:
#Save the results
df_metrics_final.to_csv('ttc_bangalore.csv', index = False)

In [10]:
df_final.shape

(4037826, 22)